In [70]:
import pandas as pd
import geopandas as gpd

In [71]:
df_prodes = pd.read_csv("./desmatamento/desmatamento_por_municipio_2023.csv")
df_biomas = pd.read_csv('impe_dados_biomas.csv.gz')
df_biomas = df_biomas.query("ano == 2023")
df_regioes = pd.read_csv('./municipios_regioes.csv')

In [72]:
df_prodes_municipios = pd.merge(df_prodes, df_regioes, right_on='codigo_municipio_ibge',left_on='id_municipio', how='inner')

In [73]:
estados_amazonia_legal = ['AC', 'AP', 'AM', 'MA', 'MT', 'PA', 'RO', 'RR', 'TO']
shapefile = "./ibge_malha_municipal/BR_Municipios_2024.shp"
gdf = gpd.read_file(shapefile)
gdf = gdf[gdf["SIGLA_UF"].isin(estados_amazonia_legal)].copy()
gdf = gdf.to_crs("EPSG:5880")
df_municipios = gdf[['CD_MUN', 'NM_MUN', 'SIGLA_UF', 'AREA_KM2']]

In [74]:
df_municipios['CD_MUN'] = df_municipios['CD_MUN'].fillna(0).astype(int) 
df_prodes['id_municipio'] = df_prodes['id_municipio'].fillna(0).astype(int) 
df_biomas['id_municipio'] = df_biomas['id_municipio'].fillna(0).astype(int) 
df_prodes['area_desmatada'] = df_prodes['area_desmatada_ha'] / 100

/var/folders/gy/cxx2_pv95ys52g0qvnjgcsqm0000gn/T/ipykernel_11853/345991298.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_municipios['CD_MUN'] = df_municipios['CD_MUN'].fillna(0).astype(int)


In [75]:
df_biomas_municipios = pd.merge(df_biomas, df_municipios, right_on='CD_MUN',left_on='id_municipio', how='inner')
df_uf_agrupado_biomas = df_biomas_municipios.groupby('SIGLA_UF')[['desmatado', 'AREA_KM2']].sum().reset_index()
df_uf_agrupado_biomas['proporcao_desmatada'] = df_uf_agrupado_biomas['desmatado'] / (df_uf_agrupado_biomas['AREA_KM2'] * 100)  

In [76]:
df_uf_agrupado_biomas

,SIGLA_UF,desmatado,AREA_KM2,proporcao_desmatada
0,AC,26220.7,164082.961,0.001598
1,AM,49279.5,1558706.131,0.000316
2,AP,3075.6,142253.880,0.000216
3,MA,165999.3,380789.258,0.004359
4,MT,369926.3,1375902.458,0.002689
5,PA,291733.8,1286634.108,0.002267
6,RO,99137.1,249462.754,0.003974
7,RR,12397.7,223505.384,0.000555
8,TO,109550.6,313171.978,0.003498


In [77]:
df_prodes_municipios = pd.merge(df_prodes, df_municipios, right_on='CD_MUN',left_on='id_municipio', how='inner')
df_uf_agrupado = df_prodes_municipios.groupby('SIGLA_UF')[['area_desmatada', 'AREA_KM2']].sum().reset_index()
df_uf_agrupado['proporcao_desmatada'] = df_uf_agrupado['area_desmatada'] / (df_uf_agrupado['AREA_KM2'] * 100)  

In [ ]:
df_estado = df_prodes_municipios.groupby('SIGLA_UF', as_index=False)['area_desmatada'].sum()
total_area_desmatada = df_estado['area_desmatada'].sum()
df_estado['percentual'] = (df_estado['area_desmatada'] / total_area_desmatada) * 100
df_estado = df_estado.sort_values(by='area_desmatada', ascending=False)
print(df_estado)

  SIGLA_UF  area_desmatada  percentual
5       PA     2887.929983   35.723892
4       MT     1943.513626   24.041397
1       AM     1327.683014   16.423530
6       RO      807.855563    9.993229
0       AC      480.346671    5.941921
3       MA      312.442703    3.864938
7       RR      278.109353    3.440232
8       TO       30.454371    0.376723
2       AP       15.694206    0.194138
